In [2]:
USE WideWorldImporters

Commands completed successfully.

Total execution time: 00:00:00.002

## Filtering Results

Something else we may notice in the Colors table is that there are lots of colors with <code>Blue</code> in their name. What are those colors?

### WHERE keyword

We can filter our SQL results using the<code>WHERE</code>keyword. Let's start with getting the record where the ColorName is equal to Blue. Its query looks like this:

In [12]:
SELECT * FROM Warehouse.Colors WHERE ColorName='Blue'

(1 row affected)

Total execution time: 00:00:00.021

ColorID,ColorName,LastEditedBy,ValidFrom,ValidTo
4,Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


We can use the <code>=</code> operator to find records that have values equal to a particular value. Since Blue is a string, you can use single quotes to surround the value.

What if you use double quotes? Try this:

In [13]:
SELECT * FROM Warehouse.Colors WHERE ColorName="Blue"

: Msg 207, Level 16, State 1, Line 1
Invalid column name 'Blue'.

Total execution time: 00:00:00.016

So you need to surround string values in quotes.

> What if the value contains quotes?  Stay tuned, as we will explain that here shortly!

Do you need quotes if the value is a number?  Let's see what color has the ColorID of 2.  We can do that with this query:

In [16]:
SELECT * FROM Warehouse.Colors WHERE ColorID=2

(1 row affected)

Total execution time: 00:00:00.010

ColorID,ColorName,LastEditedBy,ValidFrom,ValidTo
2,Beige,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


Beige happened to be the color with the ColorID of 2. We did not need to use quotes for a numeric value. However, we need it for strings.

Suppose we want colors that have an ID less than 5. We can use \< for less than, \> for greater than, and a combination of those with = to filter numeric values. For colors with IDs less than 5, that query looks like this:

In [37]:
SELECT ColorID, ColorName FROM Warehouse.Colors WHERE ColorID < 5

(4 rows affected)

Total execution time: 00:00:00.030

ColorID,ColorName
1,Azure
2,Beige
3,Black
4,Blue


If we wanted to include the value equal to 5, we could use \<= to find all colors with IDs less than or equal to 5.

In [38]:
SELECT ColorID, ColorName FROM Warehouse.Colors WHERE ColorID <= 5

(5 rows affected)

Total execution time: 00:00:00.008

ColorID,ColorName
1,Azure
2,Beige
3,Black
4,Blue
5,Charcoal


### Working with NULL


NULL is a keyword in SQL that represents the absence of a value. When working with null, there are some things to keep in mind:

- We can filter NULL values in our WHERE clause.
    - If we want to see only null values, we use IS NULL.
    - If we want to see only values and exclude the NULL values, we use IS NOT NULL.
- When there is a NULL value, you may want to use a default value in your query. We can use CASE to check a condition and assign a different value.

When we look at the columns on Sales.Customers, you can see if there are columns that allow NULL by the presence of "null" in the description. If the columns do not allow NULL values, you will see "not null" in the column description. For our NULL examples, we will be working with the CreditLimit field.

Since we may be returning a lot of records, let's look at the first 5 records. We can use this with the TOP keyword as part of the SELECT clause. First, let's look at those who have NULL credit limits. We will use IS NULL in our WHERE clause.

In [ ]:
SELECT TOP 5 CustomerName, CreditLimit FROM Sales.Customers WHERE CreditLimit IS NULL

Now let's look at the top 3 records where a CreditLimit is set. We want records where the credit limit is not null, so we will use IS NOT NULL in the WHERE clause.

In [ ]:
SELECT TOP 3 CustomerName, CreditLimit FROM Sales.Customers WHERE CreditLimit IS NOT NULL

### CASE statement

Suppose we want to show customers with their credit limits, and for those who have no limit, we want to show 9999999999. We can achieve this with the CASE statement in our SELECT clause.

The CASE statement allows us to use conditional logic to set a value.

We are only grabbing the first 3 records to keep our data small.

In [ ]:
SELECT TOP 3 CustomerName, 
CASE
    WHEN CreditLimit IS NULL THEN 9999999999
    ELSE CreditLimit
END 
FROM Sales.Customers

Notice that the CASE statement comes back without a column name. It is common practice to use an alias when working with CASE statements. Aliases for CASE statements are similar to what we saw with the EOMONTH() examples earlier in this lesson. We use the AS keyword to set the alias for the CASE results.

In [ ]:
SELECT TOP 3 CustomerName, 
CASE
    WHEN CreditLimit IS NULL THEN 9999999999
    ELSE CreditLimit
END AS CreditLimit 
FROM Sales.Customers

### CONVERT and CAST

With our example, it may make more sense to show 'No Limit' when the CreditLimit field is null. However, we cannot simply say:

```
SELECT TOP 3 CustomerName, 
CASE
   WHEN CreditLimit IS NULL THEN 'No Limit'
   ELSE CreditLimit
END AS CreditLimit 
FROM Sales.Customers
```

Remember that the CreditLimit is a numeric value, whereas 'No Limit' is a string. This is what is known as a type mismatch - column values need to have the same type throughout and cannot mix types.

CAST and CONVERT can be used to change the credit limit to a string. Before we use them let's understand why there are two keywords involved:

- CAST is ANSI SQL specific, whereas CONVERT is specific to SQL Server.
    
- CAST uses the AS keyword not as a column alias but as a type assignment operator, assigning a type to the value.
    
- CONVERT can be used for formatting, especially with date time values.
    
    We can convert values from numeric to string using the CAST or CONVERT keywords. Let's look at how we use these

In [3]:
SELECT TOP 3 CustomerName, 
CASE
   WHEN CreditLimit IS NULL THEN 'No Limit'
   ELSE CAST(CreditLimit AS varchar(20))
END AS CreditLimit 
FROM Sales.Customers

(3 rows affected)

Total execution time: 00:00:00.051

CustomerName,CreditLimit
Tailspin Toys (Head Office),No Limit
"Tailspin Toys (Sylvanite, MT)",No Limit
"Tailspin Toys (Peeples Valley, AZ)",No Limit


In [4]:
SELECT TOP 3 CustomerName, 
CASE
   WHEN CreditLimit IS NULL THEN 'No Limit'
   ELSE CONVERT(varchar(20),CreditLimit)
END AS CreditLimit 
FROM Sales.Customers

(3 rows affected)

Total execution time: 00:00:00.009

CustomerName,CreditLimit
Tailspin Toys (Head Office),No Limit
"Tailspin Toys (Sylvanite, MT)",No Limit
"Tailspin Toys (Peeples Valley, AZ)",No Limit


Since our query results do not show a good mix of those with limits and those with no limits, let's use a WHERE clause to include those we know who have limits and those we know that do not.

- For the names we know have limits, we can use the IN keyword to limit them.
- For those who have no limits - such as Tailspin Toys - we can use the LIKE keyword to filter the customer name.
- We can use the OR keyword to join those clauses. We will explain the OR keyword after we run this query.
- To ensure we get more than just the Tailspin Toys entries, we can use an ORDER BY clause with the field name we want to order by - in this case, CustomerName.  Since we are not specifying what order to use, the CustomerName will sort alphabetically, A-to-Z.

In [6]:
SELECT TOP 5 CustomerName, CASE
    WHEN CreditLimit IS NULL THEN 'No Limit'
    ELSE CAST(CreditLimit AS varchar(20))
END AS CreditLimit
FROM Sales.Customers
WHERE CustomerName IN ('Eric Torres', 'Cosmina Vlad', 'Bala Dixit') OR CustomerName LIKE 'Tailspin%'
ORDER BY CustomerName

(5 rows affected)

Total execution time: 00:00:00.026

CustomerName,CreditLimit
Bala Dixit,2000.00
Cosmina Vlad,2940.00
Eric Torres,3000.00
"Tailspin Toys (Absecon, NJ)",No Limit
"Tailspin Toys (Aceitunas, PR)",No Limit


### OR keyword

The OR keyword is used between clauses when you want one condition or another condition to be met.  In the above example, we wanted customers with the name in the collection of names in the IN clause or customers with Tailspin at the start of their name based on the LIKE clause.  As long as the name meets one of those, it will appear in the results.

Suppose our salesperson came to use wanting to see accounts that were opened in 2015 and 2016, newest account to oldest. An account opened date only has one year. We can find accounts that were opened in 2015 OR were open in 2016. The query for this is:

In [ ]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) = 2015 OR YEAR(AccountOpenedDate) = 2016 ORDER BY AccountOpenedDate DESC

### LIKE keyword

Let's go back to strings, specifically Blue. There are various shades of blue with blue in their name. We can use the LIKE operator with its<code>%</code>wildcard to search text . Suppose we want to find all ColorNames that start with Blue. The query looks like this:

In [35]:
SELECT * FROM Warehouse.Colors WHERE ColorName LIKE 'Blue%'

(1 row affected)

Total execution time: 00:00:00.034

ColorID,ColorName,LastEditedBy,ValidFrom,ValidTo
4,Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


There is one color name that starts with Blue. Suppose we want to see all ColorNames that end with Blue. That query looks like this:

In [18]:
SELECT * FROM Warehouse.Colors WHERE ColorName LIKE '%Blue'

(3 rows affected)

Total execution time: 00:00:00.008

ColorID,ColorName,LastEditedBy,ValidFrom,ValidTo
4,Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
22,Navy Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
29,Royal Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


There are three colors that end in blue - Blue, Navy Blue, and Royal Blue. What if we wanted to check for ColorNames that have Blue anywhere? That query looks like this:

In [19]:
SELECT * FROM Warehouse.Colors WHERE ColorName LIKE '%Blue%'

(3 rows affected)

Total execution time: 00:00:00.006

ColorID,ColorName,LastEditedBy,ValidFrom,ValidTo
4,Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
22,Navy Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999
29,Royal Blue,1,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999


The `%` wildcard character - a place holder for "anything" - can be used multiple times when building a pattern to use in searching text.

So:

- 'Blue%' means colors that start with Blue and anything following it.
- '%Blue' means colors that start with anything but must end in Blue
- '%Blue%' means colors that can start with anything and end with anything but must contain Blue somewhere in the value



Now that we have looked at the Color table a bit, let's learn more about our data by looking at the Sales.Customers table.  Now let's address the question we brought up earlier - what if our data contains a single quote?  When you are searching for a single quote in your data, it needs to be escaped - a way to tell the SQL engine that you mean it as a single quote.  To tell the SQL Engine that you want it as a single quote, you need to put another single quote before it.

So suppose we want to find customers with single quotes in their name.  We can use LIKE to filter our results and use double single quotes to search for a single quote.  The query to find customers with single quotes in their name is:

In [21]:
SELECT CustomerName FROM Sales.Customers WHERE CustomerName LIKE '%''%'

(1 row affected)

Total execution time: 00:00:00.022

CustomerName
"Wingtip Toys (Kapa'a, HI)"


That LIKE statement is built as follows:

- Starting and ending single quotes to surround the phrase
- % at the beginning and ending of the string to match.  The name can start and end with anything.
- '' - This is two single quotes in the middle of the LIKE match.  Two single quotes together like this represent a literal single quote.  So we want any CustomerName that contains a single quote.

There is a customer with a single quote in their name - Wingtip Toys (Kapa'a, HI).

### Date Functions

When filtering data, you may want to filter in a date range.  With the Sales.Customers table, there is an AccountOpenedDate field that tracks when the customer account has opened.  We can use the YEAR function to see what years the customers were established.  That query looks like this:

In [24]:
SELECT YEAR(AccountOpenedDate) FROM Sales.Customers

(663 rows affected)

Total execution time: 00:00:00.013

(No column name)
2013
2013
2013
2013
2013
2013
2013
2013
2013
2013


Running this query shows a lot of repeated years - 663 rows. We can use the <code>DISTINCT</code> keyword in the SELECT clause to get the unique values. So let's update the query.

In [25]:
SELECT DISTINCT YEAR(AccountOpenedDate) FROM Sales.Customers

(4 rows affected)

Total execution time: 00:00:00.009

(No column name)
2013
2014
2015
2016


Suppose we want to see which customers opened accounts in 2014. We only care about the CustomerName and AccountOpenedDate. Let's look at the following query:

In [28]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) = 2014

(14 rows affected)

Total execution time: 00:00:00.024

CustomerName,AccountOpenedDate
Tuulikki Linna,2014-01-15
Daniella Cavalcante,2014-01-17
Serdar ozden,2014-02-22
Emma Van Zant,2014-05-14
Veronika Necesana,2014-05-18
Chompoo Atitarn,2014-06-18
Dipti Shah,2014-08-03
Som Mukherjee,2014-08-25
Cuneyt Arslan,2014-08-26
Aishwarya Dantuluri,2014-09-03


### AND keyword

Suppose you have a salesperson who remembered bringing in the Dipti Shah account. That salesperson wanted to see what accounts were added in the month. Rather than looking up the date and writing a query to find all accounts in that month, we can store Dipti's account opened date in a variable and then query for all accounts added in that month, using MONTH to filter for a month, AND to allow additional filtering, and YEAR to filter the year.

SQL variables are declared with the DECLARE keyword, the variable name preceded by an at-sign<code>@</code>, and the variable type. The variable declaration may also include an assignment using the<code>=</code>operator and a value. Multiple variables can be declared with variable names and types in a single DECLARE statement.<br>

We can use a SELECT clause with the <code>=</code> operator to save a value in a variable. That looks like this:<br>

In [35]:
DECLARE @ShahAccountDate DATETIME
SELECT @ShahAccountDate = AccountOpenedDate FROM Sales.Customers WHERE CustomerName = 'Dipti Shah'
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE MONTH(AccountOpenedDate) = MONTH(@ShahAccountDate) AND YEAR(AccountOpenedDate) = YEAR(@ShahAccountDate)

(3 rows affected)

Total execution time: 00:00:00.019

CustomerName,AccountOpenedDate
Dipti Shah,2014-08-03
Som Mukherjee,2014-08-25
Cuneyt Arslan,2014-08-26


The salesperson was having a conversation with some others and wanted to know about the accounts opened in 2013 for companies that don't have Toys in their name. Tailspin Toys and Wingtip Toys both started with the Wide World Importers in 2013. The salesperson wants a list of customers who are not them.

We can use our AND clause to filter for the year AND a LIKE clause to filter the customer name. We can add the NOT keyword to say we want all customers that do not match our pattern - NOT LIKE. The query for all customer accounts opened in 2013 that don't have Toys in their name looks like this:

In [43]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) = 2013 AND CustomerName NOT LIKE '%Toys%'

(224 rows affected)

Total execution time: 00:00:00.015

CustomerName,AccountOpenedDate
Eric Torres,2013-01-01
Cosmina Vlad,2013-01-01
Bala Dixit,2013-01-01
Aleksandrs Riekstins,2013-01-01
Ratan Poddar,2013-01-01
Shi Tu,2013-01-01
Gunnar Lohmus,2013-01-01
Jackson Kolios,2013-01-01
Madhu Dwivedi,2013-01-01
Alena Kellnerova,2013-01-01


### BETWEEN keyword

The AND keyword is used for more than just joining clauses in the WHERE clause. It is also used as part of the BETWEEN clause that can appear in a WHERE clause.

The BETWEEN clause is inclusive of the starting and ending values. In the following example, we want to find customers who have credit limits between 2500.00 and 3000.00. The results will include those who have credit limits of 2500.00, those who have credit limits of 3000.00, and those who have credit limits between those values.

In [41]:
SELECT CustomerName, CreditLimit FROM Sales.Customers WHERE CreditLimit BETWEEN 2500.00 AND 3000.00

(46 rows affected)

Total execution time: 00:00:00.039

CustomerName,CreditLimit
Eric Torres,3000.00
Cosmina Vlad,2940.00
Shi Tu,3000.00
Shyam Poddar,2600.00
Libuse Valentova,2900.00
Jayanta Thakur,3000.00
Crina Grasu,2500.00
Biju Deb,2600.00
Alvin Bollinger,2900.00
Caterina Pinto,3000.00


BETWEEN can work with alphabetical ranges as well. Suppose we want customers whose names are between 'A' and 'B'. That query looks like this:

In [44]:
SELECT CustomerName FROM Sales.Customers WHERE CustomerName BETWEEN 'A' AND 'B' ORDER BY CustomerName

(32 rows affected)

Total execution time: 00:00:00.066

CustomerName
Aakriti Byrraju
Abel Spirlea
Abel Tatarescu
Abhra Ganguly
Adrian Andreasson
Adriana Pena
Agrita Abele
Agrita Kanepa
Aishwarya Dantuluri
Aive Petrov


Notice that there are no results that start with 'B'. This is because it is looking for whole strings that start with 'A' and stops at 'B'. If we had a customer named 'B', they would be included in the results. To understand how BETWEEN works with alphabetical clauses, change the starting or ending values and experiment. For example, what if we had said between 'Ar' and 'Bz'?

In [47]:
SELECT CustomerName FROM Sales.Customers WHERE CustomerName BETWEEN 'Ar' AND 'Bz' ORDER BY CustomerName

(18 rows affected)

Total execution time: 00:00:00.028

CustomerName
Arijit Bhuiyan
Arka Chatterjee
Baalaamjali Devulapalli
Bahaar Asef zade
Bala Dixit
Baran Jonsson
Be Trang
Bhaamini Palagummi
Bhaavan Rai
Bhadram Kamasamudram


The BETWEEN clause can be used with dates as well. Suppose we want to find out which customers opened their accounts in the first half of the year. We can use the BETWEEN keyword in our WHERE clause to filter our records to a date between a start and end date. We use the AND keyword to separate the beginning and ending clauses of the BETWEEN clause.

> We are referring to dates in the YYYY-MM-DD format, surrounding the values in single quotes.

The SET keyword can also be used to set a variable value.

In [30]:
DECLARE @StartDate date, @EndDate date 
SET @StartDate = '2014-01-01' -- YYYY-MM-DD format, surrounded by single quotes
SET @EndDate = '2014-06-30'

SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE AccountOpenedDate BETWEEN @StartDate AND @EndDate

(6 rows affected)

Total execution time: 00:00:00.035

CustomerName,AccountOpenedDate
Tuulikki Linna,2014-01-15
Daniella Cavalcante,2014-01-17
Serdar ozden,2014-02-22
Emma Van Zant,2014-05-14
Veronika Necesana,2014-05-18
Chompoo Atitarn,2014-06-18


The salesperson came back and wanted to find out what accounts were added after Aive Petrov's account though also in the same month and year. We could approach this as follows:

- Find when Aive Petrov's account was opened. Store that in a variable.
- Write a query where the account opened date is between Aive Petrov's update and the end of the month - once we know Aive's account opened date, we can figure out the end of the month manually.

<br>

In [49]:
DECLARE @CustomerAccountDate DATE
SELECT @CustomerAccountDate = AccountOpenedDate FROM Sales.Customers WHERE CustomerName = 'Aive Petrov' -- '2013-08-10'
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE AccountOpenedDate BETWEEN @CustomerAccountDate AND '2013-08-31'

(2 rows affected)

Total execution time: 00:00:00.031

CustomerName,AccountOpenedDate
Aive Petrov,2013-08-10
David Novacek,2013-08-15


We can also use EOMONTH to calculate the end of the month for a date. So our query would look like this:

In [50]:
DECLARE @CustomerAccountDate DATE
SELECT @CustomerAccountDate = AccountOpenedDate FROM Sales.Customers WHERE CustomerName = 'Aive Petrov' -- '2013-08-10'
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE AccountOpenedDate BETWEEN @CustomerAccountDate AND EOMONTH(@CustomerAccountDate)

(2 rows affected)

Total execution time: 00:00:00.017

CustomerName,AccountOpenedDate
Aive Petrov,2013-08-10
David Novacek,2013-08-15


EOMONTH can be used to calculate the end of a month based on a date passed in as well as an optional month shift.

- If we pass in a negative month shift, EOMONTH calculates for prior months.
- If we do not pass in a month shift, EOMONTH calculates only for the date passed in.
- If we pass in a positive month shift, EOMONTH calculates for future months.

Consider the following query. We are going to use GETDATE() to get the current date.

In [52]:
DECLARE @Today datetime = GETDATE()
SELECT @Today, EOMONTH(@Today,-1), EOMONTH(@Today), EOMONTH(@Today,1)

(1 row affected)

Total execution time: 00:00:00.006

(No column name),(No column name),(No column name),(No column name)
2020-11-12 16:36:06.567,2020-10-31,2020-11-30,2020-12-31


### AS keyword

In that last query, the column names came back as "(No column name)". When looking at results, it could be easy to get lost in nameless columns. We can use the AS keyword in our SELECT clause to assign column names to these values.

In [53]:
DECLARE @Today datetime = GETDATE()
SELECT @Today AS 'Today', EOMONTH(@Today,-1) AS 'EndOfLastMonth', EOMONTH(@Today) AS 'EndOfThisMonth', EOMONTH(@Today,1) AS 'EndOfNextMonth'

(1 row affected)

Total execution time: 00:00:00.008

Today,EndOfLastMonth,EndOfThisMonth,EndOfNextMonth
2020-11-12 16:40:31.890,2020-10-31,2020-11-30,2020-12-31


The AS keyword for aliases applies to columns and tables, as we will see in the lesson where we connect tables.  In these particular cases, the AS keyword is optional.  It can be replaced as a space between the field name and the alias.  The above query with the column aliases without the keyword would be:

In [1]:
DECLARE @Today datetime = GETDATE()
SELECT @Today 'Today', EOMONTH(@Today,-1) 'EndOfLastMonth', EOMONTH(@Today) 'EndOfThisMonth', EOMONTH(@Today,1) 'EndOfNextMonth'

(1 row affected)

Total execution time: 00:00:00.092

Today,EndOfLastMonth,EndOfThisMonth,EndOfNextMonth
2020-11-18 16:38:53.623,2020-10-31,2020-11-30,2020-12-31


### IN keyword

We can the IN keyword in our WHERE clause to filter values from within a group. Suppose we want to find the colors on the Warehouse.Colors table that have ColorIDs 1, 4, and 6. We can use the IN operator for that as well.  That query is:

In [ ]:
SELECT ColorID, ColorName FROM Warehouse.Colors WHERE ColorID IN (1, 4, 6)

We can also use IN with dates. Consider this example where we know that we want accounts opened on 2014-05-14, 2014-11-13, and 2014-12-03.

In [ ]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE AccountOpenedDate IN ('2014-05-14','2014-11-13', '2014-12-03')

### Ordering Results with the ORDER BY clause

By default, search results come back in an order that seems logical - typically in the natural order of the primary key. This means string keys are alphabetical A-Z, numeric keys 0-9, and other fields in their natural lower to higher - <strong>ascending</strong> - sort order. We can use the ORDER BY clause, after the WHERE clause, to control the order of the results.

We want to see when these accounts were opened, oldest to newest in 2014, so ascending. This query is:

In [55]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) = 2014 ORDER BY AccountOpenedDate

(14 rows affected)

Total execution time: 00:00:00.003

CustomerName,AccountOpenedDate
Tuulikki Linna,2014-01-15
Daniella Cavalcante,2014-01-17
Serdar ozden,2014-02-22
Emma Van Zant,2014-05-14
Veronika Necesana,2014-05-18
Chompoo Atitarn,2014-06-18
Dipti Shah,2014-08-03
Som Mukherjee,2014-08-25
Cuneyt Arslan,2014-08-26
Aishwarya Dantuluri,2014-09-03


The ORDER BY clause takes in a list of fields and directions. There are two directions for sorting:

- Ascending - lowest to highest, A-Z, 0-9, earliest date to latest date - considered the natural sort and does not necessarily need specified. However, you can use ASC to denote sorting ascending.
- Descending - highest to lowest, Z-A, 9-0, latest date to earliest data - noted with the DESC shortcut.

Let's order our customers of 2014 from latest to earliest accounts. For this, we will need the ORDER BY clause after the WHERE clause with AccountOpenedDate DESC to indicate we want to sort by the account opened date latest to earliest. This query looks like:

In [56]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) = 2014 ORDER BY AccountOpenedDate DESC

(14 rows affected)

Total execution time: 00:00:00.021

CustomerName,AccountOpenedDate
Damodara Trivedi,2014-12-03
David Jaramillo,2014-11-21
Erik Malk,2014-11-13
Manjunatha Karnik,2014-11-08
Aishwarya Dantuluri,2014-09-03
Cuneyt Arslan,2014-08-26
Som Mukherjee,2014-08-25
Dipti Shah,2014-08-03
Chompoo Atitarn,2014-06-18
Veronika Necesana,2014-05-18


The salesperson wants to know what accounts - besides the Toys accounts - were opened on 2013-01-01 and would like the customers listed alphabetically.

In [61]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE AccountOpenedDate = '2013-01-01' AND CustomerName NOT LIKE '%Toys%' ORDER BY CustomerName

(200 rows affected)

Total execution time: 00:00:00.019

CustomerName,AccountOpenedDate
Aakriti Byrraju,2013-01-01
Abel Spirlea,2013-01-01
Abel Tatarescu,2013-01-01
Adrian Andreasson,2013-01-01
Agrita Kanepa,2013-01-01
Ajitaab Pakalapati,2013-01-01
Akhil Nadar,2013-01-01
Aleksandrs Riekstins,2013-01-01
Alena Kellnerova,2013-01-01
Alinne Matos,2013-01-01


Now they want a list of all the accounts opened in years other than 2013, ordered by latest to earliest and alphabetical for accounts that share the same day. We can do this by having multiple fields and directions in our ORDER BY clause. As our sample data does not have this case, we can cause this to happen. We are going to update the record for Sylvie Laramee and set their account opened date to an existing date, shared with Amrita Ganguly.

In [64]:
UPDATE Sales.Customers SET AccountOpenedDate='2015-09-23' WHERE CustomerName = 'Sylvie Laramee'

(1 row affected)

Total execution time: 00:00:00.059

Now that we have customers who have the same AccountOpenedDate, let's see how we write the query to sort them latest to earliest and then alphabetically by customer name.

In [65]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) != 2013 ORDER BY AccountOpenedDate DESC, CustomerName ASC

(37 rows affected)

Total execution time: 00:00:00.018

CustomerName,AccountOpenedDate
Agrita Abele,2016-05-07
Anand Mudaliyar,2016-04-23
Jibek Juniskyzy,2016-03-13
Jaroslav Fisar,2016-02-01
Ganesh Majumdar,2016-01-09
Kalyani Benjaree,2015-12-20
Adriana Pena,2015-11-30
Emma Salpa,2015-11-29
Luis Saucedo,2015-11-27
Ian Olofsson,2015-09-26


We could also have left off the ASC keyword in the ORDER BY clause, as that is the assumed order direction if no order direction is specified.

In [66]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE YEAR(AccountOpenedDate) != 2013 ORDER BY AccountOpenedDate DESC, CustomerName

(37 rows affected)

Total execution time: 00:00:00.009

CustomerName,AccountOpenedDate
Agrita Abele,2016-05-07
Anand Mudaliyar,2016-04-23
Jibek Juniskyzy,2016-03-13
Jaroslav Fisar,2016-02-01
Ganesh Majumdar,2016-01-09
Kalyani Benjaree,2015-12-20
Adriana Pena,2015-11-30
Emma Salpa,2015-11-29
Luis Saucedo,2015-11-27
Ian Olofsson,2015-09-26


## <span style="font-size: 14px;">Our salesperson came to us with a list of people they want to find account opening dates for. This is their list:</span>  

- Nguyen Banh
- Erik Malk
- Emma Van Zandt
- Damodara Trivedi

They would like the accounts listed in the order their accounts were opened.

We can use the IN keyword in our WHERE clause to filter for values in a group of values. Let's take a look at the following statement:

In [16]:
SELECT CustomerName, AccountOpenedDate FROM Sales.Customers WHERE CustomerName IN ('Nguyen Banh', 'Erik Malk', 'Emma Van Zant', 'Damodara Trivedi') ORDER BY AccountOpenedDate

(4 rows affected)

Total execution time: 00:00:00.037

CustomerName,AccountOpenedDate
Emma Van Zant,2014-05-14
Erik Malk,2014-11-13
Damodara Trivedi,2014-12-03
Nguyen Banh,2015-01-22
